In [1]:
import pandas as pd
import numpy as np
import spacy
import nltk
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
list1 = ['I like to play computer games', 'I like to run']
list2 = ['I like playing games', 'I like walking']

In [8]:
class Evaluator:
    def __init__(self,list1,list2):
        self.list1 = list1
        self.list2 = list2
        if (len(list1) != len(list2)):
            print('Length of 2 inputs must be equal')
        
    def __preprocessing(self,text):
        """ - Preprocessing the text
            - Planning to use the tfidf vectorizer, therefore preprocessing and removing stops words are essential
        """
        text = text.lower()
        token_words = [token.text for token in nlp(text) if not token.is_stop]
        cleaned_sentences = ' '.join(token_words)
        return cleaned_sentences
    
    def precision(self, number_of_questions_to_be_answered ,threshold = 0.5, preprocessed = False):
        """
        return 3 outputs as precision, recall and Fmeasure
        number_of_questions_to_be_answered - Questions to be answered in future - {int}
        threshold - threshold value to compare cosine similarity {between 0 to 1, float}
        preprocessed - if true stop words are removed. by default flase
        """
        self.__answer_count = 0
        if preprocessed:
            self.__prelist1 = list(map(self.__preprocessing, self.list1))
            self.__prelist2 = list(map(self.__preprocessing, self.list2))
            
            for i,j in zip(self.__prelist1, self.__prelist2):
                s1 = nlp(i)
                s2 = nlp(j)
                score = s1.similarity(s2)
                if (score >= threshold):
                    self.__answer_count += 1
        else:
            for i,j in zip(self.list1,self.list2):
                w1 = nlp(i)
                w2 = nlp(j)
                score1 = w1.similarity(w2)
                if (score1 >= threshold):
                    self.__answer_count += 1
        
        self.__precision = self.__answer_count/len(self.list1)
        self.__recall = self.__answer_count/number_of_questions_to_be_answered
        self.__Fmeasure = 2*self.__precision*self.__recall / (self.__precision+self.__recall)
        return self.__precision, self.__recall, self.__Fmeasure

In [12]:
e2 = Evaluator(list1,list2)

In [13]:
e2.precision(5, preprocessed=False)

<ipython-input-8-13bea8abf9d8>:39: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  score1 = w1.similarity(w2)


(1.0, 0.4, 0.5714285714285715)